In [ ]:
from credentials import credentials

import requests
import pandas as pd
import time

In [ ]:
centers_switzerland = [
    "Zurich, Switzerland",
    "Geneva, Switzerland",
    "Basel, Switzerland",
    "Lausanne, Switzerland",
    "Bern, Switzerland",
    "Winterthur, Switzerland",
    "Lucerne, Switzerland",
    "St. Gallen, Switzerland",
    "Lugano, Switzerland",
    "Biel/Bienne, Switzerland",
    "Thun, Switzerland",
    "Köniz, Switzerland",
    "La Chaux-de-Fonds, Switzerland",
    "Schaffhausen, Switzerland",
    "Fribourg, Switzerland",
    "Chur, Switzerland",
    "Neuchâtel, Switzerland",
    "Uster, Switzerland",
    "Sion, Switzerland"
]

centers_germany = [
    "Berlin, Germany",
    "Hamburg, Germany",
    "Munich, Germany",
    "Cologne, Germany",
    "Frankfurt, Germany",
    "Stuttgart, Germany",
    "Düsseldorf, Germany",
    "Dortmund, Germany",
    "Essen, Germany",
    "Leipzig, Germany",
    "Bremen, Germany",
    "Dresden, Germany",
    "Hanover, Germany",
    "Nuremberg, Germany",
    "Duisburg, Germany",
    "Bochum, Germany",
    "Wuppertal, Germany",
    "Bielefeld, Germany",
    "Bonn, Germany",
    "Münster, Germany",
    "Karlsruhe, Germany",
    "Mannheim, Germany",
    "Augsburg, Germany",
    "Wiesbaden, Germany",
    "Gelsenkirchen, Germany",
    "Mönchengladbach, Germany",
    "Braunschweig, Germany",
    "Chemnitz, Germany",
    "Kiel, Germany",
    "Aachen, Germany",
    "Halle, Germany",
    "Magdeburg, Germany",
    "Freiburg, Germany",
    "Krefeld, Germany",
    "Lübeck, Germany",
    "Oberhausen, Germany",
    "Erfurt, Germany",
    "Mainz, Germany",
    "Rostock, Germany",
    "Kassel, Germany"
]

In [ ]:
# Google Maps API Key
API_KEY = credentials['api_key']

# Google Places and Place Details endpoint URLs
BASE_PLACES_URL = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
PLACE_DETAILS_URL = 'https://maps.googleapis.com/maps/api/place/details/json'


def get_coordinates_for_city(city_name):
    """Fetch the latitude and longitude for a given city using Google's Geocoding API."""
    print(city_name)
    
    params = {
        'address': city_name,
        'key': API_KEY
    }
    
    response = requests.get(BASE_GEOCODING_URL, params=params)
    location_data = response.json()
    print(location_data)
    location = location_data['results'][0]['geometry']['location']
    
    return location['lat'], location['lng']


def get_place_details(place_id):
    """Retrieve detailed information for a given place using its place_id."""
    
    params = {
        'place_id': place_id,
        'fields': 'name,address_component,formatted_phone_number,website',
        'key': API_KEY
    }
    
    response = requests.get(PLACE_DETAILS_URL, params=params)
    
    return response.json().get('result', {})


def get_boutiques_for_center(lat, lng):
    """Fetch boutiques near a given latitude and longitude."""
    
    boutiques = []
    params = {
        'location': f'{lat},{lng}',
        'radius': 15000,
        'type': 'store',
        'keyword': 'boutique',
        'key': API_KEY
    }
    
    response = requests.get(BASE_PLACES_URL, params=params)
    results = response.json()
    
    boutiques.extend(results.get('results', []))
    
    # Get additional results from next pages (if they exist)
    while 'next_page_token' in results:
        time.sleep(2)  # Wait a bit before next request to ensure the token is valid
        params['pagetoken'] = results['next_page_token']
        response = requests.get(BASE_PLACES_URL, params=params)
        results = response.json()
        boutiques.extend(results.get('results', []))
    
    return boutiques


def process_centers(centers):
    """Retrieve boutiques details for a list of city centers."""
    
    all_boutiques = []
    
    for center in centers:
        lat, lng = get_coordinates_for_city(center)
        boutiques = get_boutiques_for_center(lat, lng)
        
        for boutique in boutiques:
            details = get_place_details(boutique['place_id'])

            # Extracting required information
            name = details.get('name', '')
            address = ''.join([component['long_name'] + ' ' for component in details.get('address_components', []) if component['types'][0] in ['route', 'street_number']]).strip()
            postal_code = ''.join([component['long_name'] for component in details.get('address_components', []) if 'postal_code' in component['types']])
            phone_number = details.get('formatted_phone_number', '')
            website = details.get('website', '')

            all_boutiques.append({
                'Name': name,
                'Address': address,
                'PLZ': postal_code,
                'Phone Number': phone_number,
                'Website': website
            })

    return all_boutiques


# Process Switzerland and Germany centers separately and save results to Excel
df_switzerland = pd.DataFrame(process_centers(centers_switzerland)).drop_duplicates()
df_switzerland.to_excel('swiss_boutiques.xlsx', index=False)

df_germany = pd.DataFrame(process_centers(centers_germany)).drop_duplicates()
df_germany.to_excel('german_boutiques.xlsx', index=False)
